# **SQL Queries**

In [2]:
import pandas as pd
import sqlalchemy as sa
import pymysql
import mysql.connector

### At this point, I connect to the database with mysql connector and run different queries, and one final query

In [1]:
consulta_proceso="""

/*Clasificar los productos segun la familia*/

drop table if exists data.articulos_top;

create table data.articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from data.raw_ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

);


/*Crear un calendario general con una consulta recursiva del 1/1/2017 al 31/12/2023*/

drop table if exists data.calendario_dias;

create table data.calendario_dias as (

with recursive cte_calendario as (
	select date('2017-01-01') as calendar_date -- fecha de inicio
	union all
	select date_add(calendar_date, interval 1 day) as calendar_date from cte_calendario
	where date_add(calendar_date, interval 1 day) <= date('2023-12-31') -- condicion de fin, fecha final
)

select
calendar_date as fecha,
year(calendar_date) as fx_anno,
month(calendar_date) as fx_mes,
day(calendar_date) as fx_day,
date_format(calendar_date, '%Y%m') as fx_anno_mes,
date_format(calendar_Date,'%x-%v') as semana -- formato
from cte_calendario

);


/*Añadir los días festivos al calendario*/

drop table if exists data.calendario_completo;

create table data.calendario_completo as (
select
base.*,
festivos.festivo
from data.calendario_dias base
left join
	(select
	a.*,
	row_number() over(partition by a.fecha order by a.festivo) as orden
	from data.raw_calendario a
	) festivos
on base.fecha=festivos.fecha
and festivos.orden=1
);


/*Agregar las ventas por día y añadir el orden del producto y el rango de fecha de estudio*/

drop table if exists data.ventas_diarias;

create table data.ventas_diarias as (
select
base.familia,
base.tipo,
base.fechaVenta,
calendario.festivo,
base.articulo,
sum(base.precio*base.cantidad)/sum(base.cantidad) as precio, /* Se calcula como el precio promedio del día, ya que hay cambios de precio a lo largo del día*/
articulos.orden as orden_articulo_familia,
case when base.fechaVenta >=date('2021-05-01') then 'S' else 'N' end as in_fecha_estudio,
sum(base.cantidad) as cantidad,
sum(base.importe) as importe

from data.raw_ventas base

inner join data.calendario_completo calendario
on base.FechaVenta=calendario.fecha

inner join data.articulos_top articulos
on base.familia=articulos.familia
and base.articulo=articulos.articulo

group by 1,2,3,4,5,7,8);


/*Crear una vista final con los datos para el proyecto*/

drop view if exists data.ventas_diarias_estudio_completo;

create view data.ventas_diarias_estudio_completo as
(select *
from data.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and orden_articulo_familia<=5);


/*Crear una vista extra que no tenga días de mayo 23*/

drop view if exists data.ventas_diarias_estudio;

create view data.ventas_diarias_estudio as
(select *
from data.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and fechaventa < date('2023-05-01')
and orden_articulo_familia<=5);
"""

In [ ]:
conexion = mysql.connector.connect(
    host=" *** ",
    user=" *** ",
    password=" *** ",
    database= " *** "
)
cursor = conexion.cursor()

# Separate queries in a list
consultas_lista = consulta_proceso.split(";")

# Strip() remove blanks and empty elements
consultas_lista = [consulta.strip() for consulta in consultas_lista if consulta.strip()]

# Execute the query
for consulta in consultas_lista:
    cursor.execute(consulta)

# Confirm changes
conexion.commit()

### Final Query

In [7]:
class DatabaseConnection:

    def __init__(self, password,
                 dialect = "mysql",
                 driver = "pymysql",
                 username = " *** ",
                 host = " *** ",
                 port = " *** ",
                 database = " *** "):

        self.__password = password
        connection_string = f"{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}"
        self.engine = sa.create_engine(connection_string)

    # Query the database and return it as a Pandas dataframe.
    def query_to_df(self, query):
        with self.engine.connect() as conn:
            df = pd.read_sql_query(query, conn)
            return df


query = """
SELECT *

FROM ventas_diarias_estudio
WHERE ARTICULO = '3960'
"""

database = DatabaseConnection(" *** ")

# Return the above query as a Pandas dataframe
ventas = database.query_to_df(query)
ventas.head(3)

,familia,tipo,fechaVenta,festivo,articulo,precio,orden_articulo_familia,in_fecha_estudio,cantidad,importe
0,BOLLERIA,VENTA,2021-05-18,None,3960,2.318,1,S,216.0,500.688008
1,BOLLERIA,VENTA,2022-05-18,None,3960,2.591,1,S,150.0,388.650003
2,BOLLERIA,VENTA,2022-09-13,None,3960,2.591,1,S,192.0,497.472011


-----------